Note: In the end, I denormalised my data (i.e. update the `progress` when somebody votes).

See https://www.reddit.com/r/django/comments/a77bl5/sum_multiple_values_over_a_queryset_including_not/ec3x3z1

In [21]:
subquery = Subquery(
    Vote.objects
    .filter(segment=OuterRef('pk'))
    #.values('segment_id')
    #.annotate(sum=Sum('value', ))#filter=Q(**filter_fields)))
    .values('value')[:1]
)

TranslatedSegment.objects.annotate(v=subquery).aggregate(
    ct1=Count('v', filter=Q(v__gte=4)),
)

AssertionError: 

In [30]:
print(TranslatedSegment.objects.annotate(a=Sum('votes__value'), b=Sum('votes__value')).only('pk').query)

SELECT "panta_translatedsegment"."id", SUM("panta_vote"."value") AS "a", SUM("panta_vote"."value") AS "b" FROM "panta_translatedsegment" LEFT OUTER JOIN "panta_vote" ON ("panta_translatedsegment"."id" = "panta_vote"."segment_id") GROUP BY "panta_translatedsegment"."id" ORDER BY "panta_translatedsegment"."position" ASC


In [26]:
from panta.constants import IMPORTANT_HEADINGS
segments = TranslatedSegment.objects.filter(work_id=3)

headings = (
    segments.filter(tag__in=IMPORTANT_HEADINGS).annotate(
        translation_done=Subquery(
            Vote.objects.filter(
                role='translator',
                #segment__position__gte=OuterRef('position'),
                #segment__position__lt=Subquery(
                #    segments.filter(
                #        tag__in=IMPORTANT_HEADINGS,
                #        position__gt=OuterRef(OuterRef('position')),
                #    ).values_list('position', flat=True)[:1]),
            ).aggregate(sum=Sum('value')).values()[:1])
    )
)
headings[0]

TypeError: 'dict_values' object is not subscriptable

In [1]:
from django.db.models import Window, ValueRange, RowRange
from panta.constants import IMPORTANT_HEADINGS
segments = TranslatedSegment.objects.filter(work__abbreviation__iexact='cch')

In [9]:
headings = (
    segments.filter(tag__in=IMPORTANT_HEADINGS)
    .annotate(
        # _votes=Sum('votes__value'),
        translation_done=Window(
            expression=Sum("votes__value"),
            order_by=F("position").asc(),
            frame=RowRange(start=-1000, end=1000),
        )
    )
    #.order_by("position")
    .values("pk", "position", "tag", "classes", "content", "translation_done")
)
print(headings.query)

for h in headings:
    if h["translation_done"] is not None:
        print(h)
# print([h['_votes'] for h in headings])

SELECT "panta_translatedsegment"."id", "panta_translatedsegment"."position", "panta_translatedsegment"."tag", "panta_translatedsegment"."classes", "panta_translatedsegment"."content", SUM("panta_vote"."value") OVER (ORDER BY "panta_translatedsegment"."position" ASC ROWS BETWEEN 1000 PRECEDING AND 1000 FOLLOWING) AS "translation_done" FROM "panta_translatedsegment" INNER JOIN "panta_translatedwork" ON ("panta_translatedsegment"."work_id" = "panta_translatedwork"."id") LEFT OUTER JOIN "panta_vote" ON ("panta_translatedsegment"."id" = "panta_vote"."segment_id") WHERE (UPPER("panta_translatedwork"."abbreviation"::text) = UPPER(cch) AND "panta_translatedsegment"."tag" IN (h3, h2)) ORDER BY "panta_translatedsegment"."position" ASC


In [11]:
subquery = Subquery(
    Vote.objects
    .filter(segment=OuterRef('pk'))
    .values('segment_id')
    .annotate(sum=Sum('value', ))#filter=Q(**filter_fields)))
    .values('sum')[:1]
)

from panta.utils import get_vote_subquery

#headings = (segments.filter(tag__in=IMPORTANT_HEADINGS).annotate(
headings = (segments.annotate(
    _votes=Sum('votes__value'),
))

print(len(headings))
print([h._votes for h in headings])

4266
[None, None, None, None, 1, None, 1, None, None, None, None, None, None, 1, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, N

In [20]:
Vote.objects.filter(segment_id__in=segments.values_list('pk', flat=True))

<QuerySet [<Vote: +1>, <Vote: +1>, <Vote: -1>, <Vote: +1>, <Vote: +1>]>

In [62]:
segs = segments.filter(position__in=(10, 20, 30)).annotate(pos=Window(expression=Sum('position'), order_by=F('position').asc(), frame=RowRange(-2, 0)))
for s in segs:
    print(s.position, s.pos)

10 10
10 20
20 40
20 50
30 70
30 80


In [31]:
from django.db.models import Value
reset_queries()

t = (
    segments
    .filter(
        position__gte=1,
        position__lt=Subquery(
            segments
            .filter(
                tag__in=IMPORTANT_HEADINGS,
                position__gt=1000,
            )
            .values('position')
            [:1]
        ),
    )
    .annotate(
        votes_count=Sum('votes__value', filter=Q(votes__role='reviewer')),
        #trustee_votes=Sum('votes__value', filter=Q(votes__role='trustee')),
    )
    .aggregate(
        #translation_done=Count('reviewer_votes', filter=Q(reviewer_votes__gte=1)),# | Q(progress__gte=3),
        done=Count('votes_count', filter=Q(votes_count__gte=3)),
        #trustee_done=Count('trustee_votes', filter=Q(trustee_votes__gte=1)),
    )
)
t
connection.queries

[{'sql': 'SELECT COUNT("votes_count") FILTER (WHERE "__col1" >= 3) FROM (SELECT "panta_translatedsegment"."id" AS Col1, SUM("panta_vote"."value") FILTER (WHERE "panta_vote"."role" = \'reviewer\') AS "votes_count", SUM("panta_vote"."value") FILTER (WHERE "panta_vote"."role" = \'reviewer\') AS "__col1" FROM "panta_translatedsegment" INNER JOIN "panta_translatedwork" ON ("panta_translatedsegment"."work_id" = "panta_translatedwork"."id") LEFT OUTER JOIN "panta_vote" ON ("panta_translatedsegment"."id" = "panta_vote"."segment_id") WHERE (UPPER("panta_translatedwork"."abbreviation"::text) = UPPER(\'cch\') AND "panta_translatedsegment"."position" >= 1 AND "panta_translatedsegment"."position" < (SELECT U0."position" FROM "panta_translatedsegment" U0 INNER JOIN "panta_translatedwork" U1 ON (U0."work_id" = U1."id") WHERE (UPPER(U1."abbreviation"::text) = UPPER(\'cch\') AND U0."position" > 1000 AND U0."tag" IN (\'h2\', \'h3\')) ORDER BY U0."position" ASC LIMIT 1)) GROUP BY "panta_translatedsegment

In [33]:
# 2 x annotate + Window

# https://code.djangoproject.com/ticket/28296#comment:6
t = (
    segments
    .filter(
        position__gte=1,
        position__lt=Subquery(
            segments
            .filter(
                tag__in=IMPORTANT_HEADINGS,
                position__gt=1000,
            )
            .values('position')
            [:1]
        ),
    )
    .annotate(
        votes_count=Sum('votes__value', filter=Q(votes__role='reviewer')),
    )
    .annotate(
        done=Window(expression=Count('votes_count', filter=Q(votes_count__gte=3)), partition_by=F('position')),
    )
)
t

FieldError: Cannot compute Count('votes_count'): 'votes_count' is an aggregate

In [4]:
# 2 x annotate + custom Count

# https://code.djangoproject.com/ticket/28296#comment:5
class SubqueryCount(Subquery):
    output_field = IntegerField()
    def __init__(self, *args, **kwargs):
        Subquery.__init__(self, *args, **kwargs)
        self.queryset = self.queryset.annotate(cnt=Count("*")).values("cnt")
        self.queryset.query.set_group_by()

t = (
    segments
    .filter(
        position__gte=1,
        position__lt=Subquery(
            segments
            .filter(
                tag__in=IMPORTANT_HEADINGS,
                position__gt=1000,
            )
            .values('position')
            [:1]
        ),
    )
    .annotate(
        votes_count=Sum('votes__value', filter=Q(votes__role='reviewer')),
    )
    .filter(
        votes_count__gte=3,
    )
)

headings = segments.filter(tag__in=IMPORTANT_HEADINGS)
headings = headings.annotate(done=SubqueryCount(t))
print(headings.query)
for h in headings:
    print(h.done)
headings[0].done

SELECT "panta_translatedsegment"."id", "panta_translatedsegment"."created", "panta_translatedsegment"."last_modified", "panta_translatedsegment"."position", "panta_translatedsegment"."page", "panta_translatedsegment"."tag", "panta_translatedsegment"."classes", "panta_translatedsegment"."content", "panta_translatedsegment"."reference", "panta_translatedsegment"."work_id", "panta_translatedsegment"."original_id", "panta_translatedsegment"."locked_by_id", "panta_translatedsegment"."progress", (SELECT COUNT(*) AS "cnt" FROM "panta_translatedsegment" V0 INNER JOIN "panta_translatedwork" V1 ON (V0."work_id" = V1."id") LEFT OUTER JOIN "panta_vote" V2 ON (V0."id" = V2."segment_id") WHERE (UPPER(V1."abbreviation"::text) = UPPER(cch) AND V0."position" >= 1 AND V0."position" < (SELECT U0."position" FROM "panta_translatedsegment" U0 INNER JOIN "panta_translatedwork" U1 ON (U0."work_id" = U1."id") WHERE (UPPER(U1."abbreviation"::text) = UPPER(cch) AND U0."position" > 1000 AND U0."tag" IN (h3, h2)) 

ProgrammingError: missing FROM-clause entry for table "V1"
LINE 1: ..., 'h2')) GROUP BY "panta_translatedsegment"."id", "V1"."abbr...
                                                             ^


In [3]:
t = Subquery(
    segments
    .filter(
        position__gte=1,
        position__lt=Subquery(
            segments
            .filter(
                tag__in=IMPORTANT_HEADINGS,
                position__gt=1000,
            )
            .values('position')
            [:1]
        ),
    )
    .annotate(
        votes_count=Sum('votes__value', filter=Q(votes__role='reviewer')),
    )
    .filter(
        votes_count__gte=3,
    )
    .annotate(
        done=Window(expression=Count('pk'))
    )
    .values('done')
    [:1],
    output_field=IntegerField(),
)

headings = segments.filter(tag__in=IMPORTANT_HEADINGS)
headings = headings.annotate(done=t)
print(headings.query)
for h in headings:
    print(h.done)

SELECT "panta_translatedsegment"."id", "panta_translatedsegment"."created", "panta_translatedsegment"."last_modified", "panta_translatedsegment"."position", "panta_translatedsegment"."page", "panta_translatedsegment"."tag", "panta_translatedsegment"."classes", "panta_translatedsegment"."content", "panta_translatedsegment"."reference", "panta_translatedsegment"."work_id", "panta_translatedsegment"."original_id", "panta_translatedsegment"."locked_by_id", "panta_translatedsegment"."progress", (SELECT COUNT(V0."id") OVER () AS "done" FROM "panta_translatedsegment" V0 INNER JOIN "panta_translatedwork" V1 ON (V0."work_id" = V1."id") LEFT OUTER JOIN "panta_vote" V2 ON (V0."id" = V2."segment_id") WHERE (UPPER(V1."abbreviation"::text) = UPPER(cch) AND V0."position" >= 1 AND V0."position" < (SELECT U0."position" FROM "panta_translatedsegment" U0 INNER JOIN "panta_translatedwork" U1 ON (U0."work_id" = U1."id") WHERE (UPPER(U1."abbreviation"::text) = UPPER(cch) AND U0."position" > 1000 AND U0."tag

ProgrammingError: missing FROM-clause entry for table "V1"
LINE 1: ..., 'h2')) GROUP BY "panta_translatedsegment"."id", "V1"."abbr...
                                                             ^


In [61]:
from django.db import connection, reset_queries
reset_queries()
a = (
    Vote.objects
    .filter(
        role='reviewer',
        segment__position__gte=1,
        segment__position__lt=Subquery(
            segments
            .filter(
                tag__in=IMPORTANT_HEADINGS,
                position__gt=1000,
            )
            .values('position')
            [:1]
        ),
    )
    .aggregate(sum=Sum('value'))
)
#str(a.query)
connection.queries
#a

[{'sql': 'SELECT SUM("panta_vote"."value") AS "sum" FROM "panta_vote" INNER JOIN "panta_translatedsegment" ON ("panta_vote"."segment_id" = "panta_translatedsegment"."id") WHERE ("panta_vote"."role" = \'reviewer\' AND "panta_translatedsegment"."position" >= 1 AND "panta_translatedsegment"."position" < (SELECT U0."position" FROM "panta_translatedsegment" U0 INNER JOIN "panta_translatedwork" U1 ON (U0."work_id" = U1."id") WHERE (UPPER(U1."abbreviation"::text) = UPPER(\'cch\') AND U0."position" > 1000 AND U0."tag" IN (\'h2\', \'h3\')) ORDER BY U0."position" ASC LIMIT 1))',
  'time': '0.006'}]

## SQL including a subquery
https://www.eversql.com/sql-syntax-check-validator/

```mysql
SELECT
  "panta_translatedsegment"."id",
  "panta_translatedsegment"."created",
  "panta_translatedsegment"."last_modified",
  "panta_translatedsegment"."position",
  "panta_translatedsegment"."page",
  "panta_translatedsegment"."tag",
  "panta_translatedsegment"."classes",
  "panta_translatedsegment"."content",
  "panta_translatedsegment"."reference",
  "panta_translatedsegment"."work_id",
  "panta_translatedsegment"."original_id",
  "panta_translatedsegment"."locked_by_id",
  "panta_translatedsegment"."progress",
  (
    SELECT
      SUM(U0."value") FILTER (
        WHERE
          U0."role" = "translator"
      ) AS "sum"
    FROM
      "panta_vote" U0
    WHERE
      U0."segment_id" = ("panta_translatedsegment"."id")
    GROUP BY
      U0."segment_id"
    LIMIT
      1
  ) AS "translators_vote"
FROM
  "panta_translatedsegment"
ORDER BY
  "panta_translatedsegment"."position" ASC
```

In [58]:
sql = """
SELECT
  SUM("panta_vote"."value") AS "sum"
FROM
  "panta_vote"
  INNER JOIN "panta_translatedsegment" ON (
    "panta_vote"."segment_id" = "panta_translatedsegment"."id"
  )
WHERE
  (
    "panta_vote"."role" = 'reviewer'
    AND "panta_translatedsegment"."position" >= ("panta_translatedsegment"."position")
    AND "panta_translatedsegment"."position" < (
      SELECT
        U0."position"
      FROM
        "panta_translatedsegment" U0
        INNER JOIN "panta_translatedwork" U1 ON (U0."work_id" = U1."id")
      WHERE
        (
          UPPER(U1."abbreviation" :: text) = UPPER('cch')
          AND U0."position" > ("panta_translatedsegment"."position")
          AND U0."tag" IN ('h2', 'h3')
        )
      ORDER BY
        U0."position" ASC
      LIMIT
        1
    )
  )
"""

"""
SELECT
  SUM("panta_vote"."value") AS "sum"
FROM
  "panta_vote"
  INNER JOIN "panta_translatedsegment" ON (
    "panta_vote"."segment_id" = "panta_translatedsegment"."id"
  )
WHERE
  (
    "panta_vote"."role" = 'reviewer'
    AND "panta_translatedsegment"."position" >= 1
    AND "panta_translatedsegment"."position" < (
      SELECT
        U0."position"
      FROM
        "panta_translatedsegment" U0
        INNER JOIN "panta_translatedwork" U1 ON (U0."work_id" = U1."id")
      WHERE
        (
          UPPER(U1."abbreviation" :: text) = UPPER('cch')
          AND U0."position" > 1000
          AND U0."tag" IN ('h2', 'h3')
        )
      ORDER BY
        U0."position" ASC
      LIMIT
        1
    )
  )
"""
s = segments.raw(sql)
#for i in s:
#    print(i)
from django.db.models.expressions import RawSQL
    
r = (
    segments
    .filter(tag__in=IMPORTANT_HEADINGS)
    .order_by('position')
    .annotate(
        translation_done=RawSQL(sql, ())
        
    )
)
str(r.query)

'SELECT "panta_translatedsegment"."id", "panta_translatedsegment"."created", "panta_translatedsegment"."last_modified", "panta_translatedsegment"."position", "panta_translatedsegment"."page", "panta_translatedsegment"."tag", "panta_translatedsegment"."classes", "panta_translatedsegment"."content", "panta_translatedsegment"."reference", "panta_translatedsegment"."work_id", "panta_translatedsegment"."original_id", "panta_translatedsegment"."locked_by_id", "panta_translatedsegment"."progress", (\nSELECT\n  SUM("panta_vote"."value") AS "sum"\nFROM\n  "panta_vote"\n  INNER JOIN "panta_translatedsegment" ON (\n    "panta_vote"."segment_id" = "panta_translatedsegment"."id"\n  )\nWHERE\n  (\n    "panta_vote"."role" = \'reviewer\'\n    AND "panta_translatedsegment"."position" >= ("panta_translatedsegment"."position")\n    AND "panta_translatedsegment"."position" < (\n      SELECT\n        U0."position"\n      FROM\n        "panta_translatedsegment" U0\n        INNER JOIN "panta_translatedwork" 

In [59]:
r[0].translation_done

470

In [49]:
Vote.objects.filter(segment_id__in=segments.values_list('pk', flat=True), role='reviewer').count()

162

In [75]:
# Get the SQL of a subquery
from panta.utils import get_vote_subquery

q = TranslatedSegment.objects.annotate(
    translators_vote=get_vote_subquery('translator')
)
str(q.query)

'SELECT "panta_translatedsegment"."id", "panta_translatedsegment"."created", "panta_translatedsegment"."last_modified", "panta_translatedsegment"."position", "panta_translatedsegment"."page", "panta_translatedsegment"."tag", "panta_translatedsegment"."classes", "panta_translatedsegment"."content", "panta_translatedsegment"."reference", "panta_translatedsegment"."work_id", "panta_translatedsegment"."original_id", "panta_translatedsegment"."locked_by_id", "panta_translatedsegment"."progress", (SELECT SUM(U0."value") FILTER (WHERE U0."role" = translator) AS "sum" FROM "panta_vote" U0 WHERE U0."segment_id" = ("panta_translatedsegment"."id") GROUP BY U0."segment_id" LIMIT 1) AS "translators_vote" FROM "panta_translatedsegment" ORDER BY "panta_translatedsegment"."position" ASC'

In [2]:
# Try an easier get_vote_subquery
from django.db.models import IntegerField
from django.db import connection, reset_queries
reset_queries()

class SubquerySum(Subquery):
    output_field = IntegerField()
    exx = []#['panta_translatedsegment']
    #def __init__(self, *args, **kwargs):
    #    Subquery.__init__(self, *args, **kwargs)
    #    self.queryset = self.queryset.annotate(sm=Sum("value")).values("sm")
    #    self.queryset.query.set_group_by()
        
    #def set_source_expressions(self, exprs):
    #    self.exx = exprs
        
    #def get_source_expressions(self):
    #    print(self.exx)
    #    return self.exx
        


def get_vote_subquery2(role, user=None):
    filter_fields = {'role': role}
    if user:
        filter_fields.update({'user': user})
    query = Vote.objects.filter(segment=OuterRef('pk'), **filter_fields)
    #return subquery
    subquery = Subquery(
        Vote.objects.filter(segment=OuterRef('pk'), **filter_fields)
        .values('segment_id')
        .annotate(sum=Sum('value'))
        .values('sum')[:1],
        output_field=IntegerField(),
    )
    return subquery

'''
aggregates = {}
required = REQUIRED_APPROVALS[self.language]
for i, h in enumerate(headings, start=1):
    filters = {'position__gte': h['position']}
    try:
        filters['position__lt'] = headings[i]['position']
    except IndexError:
        # The last item
        pass
    aggregates['translation_done_{}'.format(i)] = Count(
        'progress',
        filter=Q(**filters)# & (
            #Q(progress__gte=TRANSLATION_DONE)
            #| Q(reviewers_vote__gte=1)
        #)
    )
    aggregates['review_done_{}'.format(i)] = Count(
        'reviewers_vote',
        #filter=Q(**filters),
        filter=Q(reviewers_vote__gte=required['reviewer'], **filters),
    )
    aggregates['trustee_done_{}'.format(i)] = Count(
        'trustees_vote',
        filter=Q(**filters),
        #filter=Case(When(trustees_vote__gte=required['trustee'], then=1, **filters))
    )
'''
try:
    aggregates = (
        segments
        .annotate(
            reviewers_vote=get_vote_subquery2('reviewer'),
            trustees_vote=get_vote_subquery2('trustee'),
        )
        .aggregate(
            translation_done=Count('progress'),
            review_done=Count('reviewers_vote', filter=Q(reviewers_vote__gte=3, position__gte=20)),
        )
    )


    aggregates
except:
    raise
    pass

print(connection.queries)


AssertionError: [Ref(__col3, Col(U0, panta.Vote.role)), Ref(__col4, Col(U0, panta.Vote.segment))]

In [14]:
# 2 x annotate + custom Count according to reddit

# https://code.djangoproject.com/ticket/28296#comment:5
class SubqueryCount(Subquery):
    output_field = IntegerField()
    def __init__(self, *args, **kwargs):
        Subquery.__init__(self, *args, **kwargs)
        self.queryset = self.queryset.annotate(cnt=Count("*")).values("cnt")
        self.queryset.query.set_group_by()

# https://www.reddit.com/r/django/comments/a77bl5/sum_multiple_values_over_a_queryset_including_not/ec1kfs7
class SQCount(Subquery):
    template = "(SELECT count(*) FROM (%(subquery)s) _count)"
    output_field = IntegerField()
        
class SQSum(Subquery):
    template = "(SELECT sum(*) FROM (%(subquery)s) _sum)"
    output_field = IntegerField()

t = (
    segments
    .filter(
        position__gte=1,
        #position__lt=Subquery(
        #    segments
        #    .filter(
        #        tag__in=IMPORTANT_HEADINGS,
        #        position__gt=1000,
        #    )
        #    .values('position')
        #    [:1]
        #),
        votes__role='reviewer',
    )
    .values('votes__value')
    #.annotate(
    #    votes_count=Sum('votes__value'),
    #)
    #.filter(
    #    votes_count__gte=3,
    #)
    #.values('votes_count')
)

headings = segments.filter(tag__in=IMPORTANT_HEADINGS)
headings = headings.annotate(done=SQCount(SQSum(t).filter(_count__gte=3)))
print(headings.query)
for h in headings:
    print(h.done)
headings[0].done

AttributeError: 'SQSum' object has no attribute 'filter'

In [ ]:
    class SQCount(Subquery):
        template = "(SELECT count(*) FROM (%(subquery)s) _count)"
        output_field = IntegerField()

    class SQSum(Subquery):
        template = "(SELECT sum(*) FROM (%(subquery)s) _sum)"
        output_field = IntegerField()

    t = (
        segments
        .filter(
            position__gte=1,
            #position__lt=Subquery(
            #    segments
            #    .filter(
            #        tag__in=IMPORTANT_HEADINGS,
            #        position__gt=1000,
            #    )
            #    .values('position')
            #    [:1]
            #),
            votes__role='reviewer',
        )
        .values('votes__value')
        #.annotate(
        #    votes_count=Sum('votes__value'),
        #)
        #.filter(
        #    votes_count__gte=3,
        #)
        #.values('votes_count')
    )

    headings = segments.filter(tag__in=IMPORTANT_HEADINGS)
    headings = headings.annotate(done=SQCount(t))

In [13]:
t = (
    segments
    .filter(
        position__gte=1,
        #position__lt=Subquery(
        #    segments
        #    .filter(
        #        tag__in=IMPORTANT_HEADINGS,
        #        position__gt=1000,
        #    )
        #    .values('position')
        #    [:1]
        #),
        votes__role='reviewer',
    )
    .values('votes__value')
    #.annotate(
    #    votes_count=Sum('votes__value'),
    #)
    #.filter(
    #    votes_count__gte=3,
    #)
    #.values('votes_count')
)
t

<QuerySet [{'votes__value': 1}, {'votes__value': 1}, {'votes__value': 1}, {'votes__value': 1}, {'votes__value': 1}, {'votes__value': -1}, {'votes__value': 1}, {'votes__value': -1}, {'votes__value': 1}, {'votes__value': 1}, {'votes__value': -1}, {'votes__value': 1}, {'votes__value': 1}, {'votes__value': 1}, {'votes__value': 1}, {'votes__value': 1}, {'votes__value': 1}, {'votes__value': 1}, {'votes__value': 1}, {'votes__value': -1}, '...(remaining elements truncated)...']>